In [18]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')

# Launch DPS for mapBoreal.R

In [19]:
import os
import geopandas
import pandas as pd
import json

def local_to_s3(url, user='lduncanson'):
    ''' A Function to convert local paths to s3 urls'''
    return url.replace('/projects/my-private-bucket', f's3://maap-ops-workspace/{user}')

In [33]:
atl08_filt_tindex_master =   pd.read_csv('s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv')
topo_tindex_master =         pd.read_csv('s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Topo_tindex_master.csv')
landsat_tindex_master =      pd.read_csv('s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Landsat_tindex_master.csv')

# Convert al local_paths to s3
#.. for data produced by 'lduncanson' workspace
atl08_filt_tindex_master['s3'] = [local_to_s3(local_path, user='lduncanson') for local_path in atl08_filt_tindex_master['local_path']]

#.. for data produced by 'nathanmthomas' workspace
for tindex_master in [topo_tindex_master, landsat_tindex_master]:
    tindex_master['s3'] = [local_to_s3(local_path, user='nathanmthomas') for local_path in tindex_master['local_path']]

# Use the ATL08 filtered tindex master list to tell you which tiles you'll run

In [34]:
INPUT_TILE_NUM_LIST = atl08_filt_tindex_master['tile_num'].values.astype(int).tolist()
print(len(INPUT_TILE_NUM_LIST))

# Remove duplicate tile nums
INPUT_TILE_NUM_LIST = list(set(INPUT_TILE_NUM_LIST))

print(len(INPUT_TILE_NUM_LIST))

3972
3937


In [35]:
# Check retrieval of s3 path with a tle_num
in_atl08_s3 =  atl08_filt_tindex_master['s3'].loc[atl08_filt_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
in_topo_s3 = topo_tindex_master['s3'].loc[topo_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
in_landsat_s3 = landsat_tindex_master['s3'].loc[landsat_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
print(in_atl08_s3)
print(in_topo_s3)
print(in_landsat_s3)

s3://maap-ops-workspace/lduncanson/dps_output/run_tile_atl08_ubuntu/master/2021/09/25/08/37/36/740304/atl08_004_30m_filt_topo_landsat_20210925_0009.csv
s3://maap-ops-workspace/nathanmthomas/dps_output/do_topo_stack_3-1-5_ubuntu/ops/2021/09/15/18/10/34/658640/Copernicus_9_covars_cog_topo_stack.tif
s3://maap-ops-workspace/nathanmthomas/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/09/14/19/20/02/503587/Landsat8_9_comp_cog_2015-2020_dps.tif


## Get file for boreal biomass models


In [36]:
bio_models_s3 = 'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/lduncanson/my-private-bucket/bio_models.tar'

## Run a DPS job

In [37]:
RUN_DPS  = True

if RUN_DPS:
    ##################################
    #Test DPS submission on a single file
    for i, INPUT_TILE_NUM in enumerate(INPUT_TILE_NUM_LIST[0:1]):
        DPS_num = i+1
        
        # Get the s3 paths of the corresponding input filenames with an input tile_num
        in_atl08_s3 =  atl08_filt_tindex_master['s3'].loc[atl08_filt_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_topo_s3 = topo_tindex_master['s3'].loc[topo_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_landsat_s3 = landsat_tindex_master['s3'].loc[landsat_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        
        #print(in_atl08_s3) 
        #print(in_topo_s3)
        #print(in_landsat_s3)
        
        if True:
            in_param_dict = {
                                    'in_atl08_fn': in_atl08_s3,
                                    'in_topo_fn': in_topo_s3,
                                    'in_landsat_fn': in_landsat_s3,
                                    'in_models_tar': bio_models_s3
                }

            submit_result = maap.submitJob(
                    identifier='run_boreal_biomass',
                    algo_id='run_boreal_biomass_ubuntu',
                    version='master',
                    username='lduncanson', # username needs to be the same as whoever created the workspace
                    queue='maap-dps-worker-16gb',
                    **in_param_dict
                )

            #submit_result = 'submit test'
            if DPS_num in [1, 10, 50, 100,200, 500,1000,3000,4000, len(INPUT_TILE_NUM_LIST)]:
                print(f"DPS run num: {DPS_num}, tile num: {INPUT_TILE_NUM}, job info: {submit_result}") 
                print(f"DPS job status: {maap.getJobStatus(submit_result.get('job_id')) }" )
 

s3://maap-ops-workspace/lduncanson/dps_output/run_tile_atl08_ubuntu/master/2021/09/25/08/37/36/740304/atl08_004_30m_filt_topo_landsat_20210925_0009.csv
DPS run num: 1, tile num: 9, job info: {'status': 'success', 'http_status_code': 200, 'job_id': '9705cfce-85d1-446b-bb96-fab8f7880787'}
DPS job status: <Response [200]>
